# **Малахов Дмитрий, Вариант 1**

# **Иногда очень круто знать что где находиться и поэтому время на поиск на кр уменьшается кратно** 

# **Библиотеки**

In [5]:
import collections
import random

# **1**

№1 Oбъясните принцип работы фильтра Калмана.

Фильтр Калмана — это алгоритм оценивания скрытого состояния в линейной стохастической системе (LSS), где динамика и наблюдения заданы двумя матричными уравнениями - коротко четко и ясно 

## LSS-модель

Модель записывается так (матрицы могут зависеть от $t$): 
- Переход состояния:
  $
  x_t = A_t x_{t-1} + C_t v_t,
  $
  где $x_t$ — скрытое состояние, $v_t \sim N(0, I)$, ковариация шума процесса равна $C_t C_t'$

- Измерение:
  $
  y_t = G_t x_t + H_t w_t,
  $
  где $y_t$ — наблюдение, $w_t \sim N(0, I)$, ковариация шума измерения $H_t H_t'$ 

Шумы независимы, всё стохастическое содержится именно в этих двух уравнениях - Это база 

## Два шага на каждом $t$

Начинаем с априорных $x_{0|0}$ и $\Sigma_{xx,0|0}$ 

### 1. Прогнозирование (Делай раз)

До прихода $y_t$ строятся прогнозы:

- состояние:
  $
  x_{t|t-1} = A_t x_{t-1|t-1},
  $
- наблюдение:
  $
  y_{t|t-1} = G_t x_{t|t-1},
  $
- ковариация состояния:
  $
  \Sigma_{xx,t|t-1} = A_t \Sigma_{xx,t-1|t-1} A_t' + C_t C_t',
  $
- ковариация наблюдения:
  $
  \Sigma_{yy,t|t-1} = G_t \Sigma_{xx,t|t-1} G_t' + H_t H_t',
  $
- кросс-ковариация:
  $
  \Sigma_{yx,t|t-1} = G_t \Sigma_{xx,t|t-1}.
  $

Это даёт «лучший» прогноз будущего и его неопределённость 

### 2. Коррекция (Делай два)
 
После наблюдения $y_t$:

- инновация:
  $
  e_t = y_t - y_{t|t-1},
  $
- Kalman gain:
  $
  K_t = \Sigma_{xy,t|t-1} (\Sigma_{yy,t|t-1})^{-1},
  $
- обновлённое состояние:
  $
  x_{t|t} = x_{t|t-1} + K_t e_t,
  $
- обновлённая ковариация:
  $
  \Sigma_{xx,t|t} = \Sigma_{xx,t|t-1} - K_t \Sigma_{yy,t|t-1} K_t'.
  $

Если измерения очень шумные, $\Sigma_{yy}$ большая, $K_t$ маленький — доверяем модели; если процесс сильно шумный, важнее данные. 

## Сглаживание (Делай три )

После прямого прохода можно сделать обратный (smoothing), улучшив оценки прошлых состояний, используя данные до $T$:

- матрица:
  $
  J_t = \Sigma_{xx,t|t} A_{t+1}' (\Sigma_{xx,t+1|t})^{-1},
  $
- сглаженное состояние:
  $
  x_{t|T} = x_{t|t} + J_t (x_{t+1|T} - x_{t+1|t}),
  $
- сглаженная ковариация:
  $
  \Sigma_{xx,t|T} = \Sigma_{xx,t|t} + J_t (\Sigma_{xx,t+1|T} - \Sigma_{xx,t+1|t}) J_t'.
  $

Этот backward pass даёт более гладкие ретроспективные оценки траектории состояния 


## Мини ревью 

Фильтр Калмана оценивает скрытое состояние линейной стохастической системы, заданной уравнением перехода и уравнением измерения с гауссовскими шумами. На каждом шаге он делает прогноз состояния и наблюдения, обновляет их ковариации, а затем корректирует прогноз с помощью нового измерения через Kalman gain, балансируя доверие к модели и данным. После прямого прохода можно выполнить сглаживание назад по времени, чтобы улучшить оценки прошлых состояний, используя всю доступную выборку.



# **№2**

На одном из занятий рассматривался метод, который назывался Value Function Iteration. Поясните
1. что это за метод;
2. для решения какой из задач мы его применяли.



## 1. Общая идея VFI

Нами рассматривается динамическая задача, где в каждой точке состояния $v$ нужно принять решение (выбрать действие или следующую вершину лучше смотреть контекст), заплатить текущую цену и перейти дальше.

Такде у нас есть

**Функция ценности** $J(v)$ которая показывает минимальную суммарную стоимость (или максимальную выгоду), если стартовать из состояния $v$ и дальше действовать оптимально.

Уравнение Беллмана в минимизационной постановке выглядит так (брал из семинаров):

$
J(v) = \min_{x \in \text{доступных соседях}} \{ c(v, x) + J(x) \},
$

где $c(v,x)$ — стоимость перехода из $v$ в соседний узел $x$

Проблема: $J(v)$ неизвестна, и формула использует саму себя. VFI решает это **итерациями**.

## 2. Как работает алгоритм пошагово... я щас устану это писать 

метод реализован как итерация над массивом $J$:

```
J = np.zeros_like(nodes, dtype=int)      # начальное приближение
nextJ = np.zeros_like(nodes, dtype=int)

for _ in range(iters):
    for v in nodes:
        nextJ[v] = np.min(Q[v, :] + J)
    if np.equal(nextJ, J).all():
        break
    J = nextJ
```


Идея каждого шага (тяжело...)

- **Шаг 0. Инициализация.**  
  Задаётся начальное приближение $J^{(0)}(v)$, в примере — просто нули. 

- **Шаг 1. Обновление по Беллману.**  
  Для каждого узла $v$ считаем новое значение:

  $
  J^{(k+1)}(v) = \min_{x} \{ Q(v,x) + J^{(k)}(x)\},
  $

  где $Q(v,x)$ — матрица стоимостей перехода из $v$ в $x$ 
  В коде это ровно строка:
  ```
  nextJ[v] = np.min(Q[v, :] + J)
  ```

  Здесь `Q[v, :] + J` — это вектор $\{ Q(v,x) + J(x)\}$ по всем возможным $x$ 

- **Шаг 2. Критерий остановки.**  
  Если $J^{(k+1)} = J^{(k)}$ (или отличаются меньше заданного порога), считаем, что нашли фиксированную точку уравнения Беллмана — функция ценности сошлась. Так проще 

- **Шаг 3. Восстановление оптимальной политики/пути.**  
  Когда есть финальный $J(v)$, оптимальное действие в $v$— это тот сосед $x$, который достигает минимума $Q(v,x) + J(x)$
  В задаче кратчайшего пути это используется так:

  ```
  currentnode = 0
  destinationnode = 6
  path = []
  while currentnode != destinationnode:
      path.append(currentnode)
      nextnode = np.argmin(Q[currentnode, :] + J)
      currentnode = int(nextnode)
  path.append(destinationnode)
  ```

  Получается путь 0 → 2 → 5 → 6. 

## 3. Конкретно: какую задачу решали 

На семинаре VFI использовался в постановке **поиска кратчайшего пути в графе** как задача динамического программирования.

- Узлы графа — это состояния $v$. 
- Матрица $Q$ содержит веса рёбер: $Q(v,x) =$ длина ребра $v \to x$ (или $\infty$, если ребра нет)
- Функция ценности $J(v)$ — минимальная длина пути от узла $v$ до целевого узла
- Уравнение Беллмана: «оптимальное расстояние из $v$ — это минимальная по всем соседям $x$ сумма: длина ребра плюс оптимальное расстояние от $x$» 


## Мини-резюме

Value Function Iteration решает динамическую задачу через уравнение Беллмана, где функция ценности $J(v)$ даёт минимальную суммарную стоимость при оптимальном поведении из состояния $v$. Алгоритм начинается с начального приближения $J^{(0)}$, затем многократно обновляет его по правилу $J^{(k+1)}(v) = \min_x \{c(v,x) + J^{(k)}(x)\}$ до сходимости и после этого восстанавливает оптимальный путь, выбирая в каждом узле соседа $x$, дающего минимум. В семинаре этот метод применялся к задаче кратчайшего пути в графе, где состояния — вершины, $Q(v,x)$ — длины рёбер, а $J(v)$— минимальное расстояние до целевой вершины. 


# **№3**

1. Опишите базовую постановку модели таблиц «затраты-выпуск».
2. Выпишите технологическую функцию в этой модели и поясните ее
смысл.

Модель «затраты–выпуск» Леонтьева описывает экономику как систему отраслей, которые покупают друг у друга промежуточные продукты, и из-за этого общий выпуск определяется линейной системой. Ключевая штука — матрица технологических коэффициентов $A$ и уравнение баланса $x = Ax + d$

## 1) Базовая постановка «затраты–выпуск»

- Есть $n$ отраслей, вектор $x$ — **валовой выпуск** по отраслям (сколько всего произвели, включая то, что пойдёт другим отраслям)
- Матрица $A$ — технологические коэффициенты (сколько продукции отрасли $i$ нужно как промежуточный ресурс на 1 единицу выпуска отрасли $j$)
- Вектор $d$ — **конечный спрос** (домохозяйства/гос-во/экспорт и т.п.)
- Баланс записывается как:  
  $
  x = Ax + d \quad \Leftrightarrow \quad (I-A)x = d,
  $
  то есть выпуск должен покрыть и промежуточный спрос $Ax$, и конечный спрос $d$. 
- Если матрица $B = I-A$ обратима, то решение:  
  $
  x = (I-A)^{-1} d,
  $
  где $(I-A)^{-1}$ в материалах прямо считается как «Leontief inverse». 

## 2) Технологическая функция и смысл
Технологическая функция в этой модели (в «количественной» версии) — это отображение конечного спроса в необходимый валовой выпуск:  
$
x(d) = (I-A)^{-1} d. 
$


Смысл такой: чтобы удовлетворить конечный спрос $d$, нужно произвести больше, чем $d$, потому что часть выпуска уйдёт на **промежуточные затраты** между отраслями (это как раз $Ax$), и именно поэтому появляется множитель $(I-A)^{-1}$.

## Мини-мини резюме 

Модель Леонтьева «затраты–выпуск»: $x = Ax + d$, где $x$ — валовой выпуск, $A$ — матрица техкоэффициентов (промежуточные затраты), $d$ — конечный спрос. Решение $x = (I-A)^{-1}d$. 

**Технологическая функция** $x(d) = (I-A)^{-1}d$ показывает, сколько всего надо произвести, чтобы покрыть конечный спрос с учётом цепочки межотраслевых поставок.




# **№4 Самое интересное, а не бесконечный латех, да прибудет творчество в сжатые сроки!**

Реализуйте алгоритм нахождения решения игры «Пятнашки».
1. Опишите, как вы будете искать решение;
2. Напишите код;
3. Покажите, что он работает.
Поскольку сложность задачи растет очень быстро, то если у вас нет под
рукой суперкомпьютера, ограничьтесь полем 3 × 3.

## **В этот раз пояснялки буду писать в облачка так как через # писать дольше и не забываем, что мы эстэты**

In [6]:
"""
РЕШЕНИЕ ИГРЫ ПЯТНАШКИ 3×3 С ИСПОЛЬЗОВАНИЕМ BFS

Алгоc:
- BFS обеспечивает нахождение кратчайшего пути
- Проверка разрешимости через инверсии
- Состояния кодируются кортежами для хэширования
- Целевое состояние: [1,2,3,4,5,6,7,8,0]  
"""

def is_solvable(state):
    """
    Проверяет разрешимость головоломки
    
    Для 3×3: число инверсий должно быть четным (очевидно)
    Инверсия = пара чисел где большее стоит раньше меньшего 
    """
    inversions = 0
    state_without_zero = [x for x in state if x != 0]
    
    for i in range(len(state_without_zero)):
        for j in range(i + 1, len(state_without_zero)):
            if state_without_zero[i] > state_without_zero[j]:
                inversions += 1
    
    return inversions % 2 == 0

def solve_puzzle(start):
    """
    Решает головоломку методом BFS
    
    Параметры:
    - start: список/кортеж из 9 чисел (0-8)
    
    Возвращает:
    - (путь решения, количество_ходов) или (None, None)
    
    BFS должен гарантировать оптимальность решения
    """
    goal = (1, 2, 3, 4, 5, 6, 7, 8, 0)
    
    if isinstance(start, list):
        start = tuple(start)
    
    if len(start) != 9 or set(start) != set(range(9)):
        return None, None
    
    if start == goal:
        return [start], 0
    
    if not is_solvable(start):
        return None, None
    
    """
    Очередь: (текущее_состояние, путь, количество_шагов)
    Посещенные: множество для O(1) проверки
    """
    queue = collections.deque([(start, [start], 0)])
    visited = {start}
    
    while queue:
        state, path, steps = queue.popleft()
        zero_pos = state.index(0)
        
        """
        Определяем возможные ходы для пустой клетки
        Проверяем границы поля 3×3
        """
        row = zero_pos // 3
        col = zero_pos % 3
        
        moves = []
        if row > 0:
            moves.append(-3)  # вверх
        if row < 2:
            moves.append(3)   # вниз
        if col > 0:
            moves.append(-1)  # влево
        if col < 2:
            moves.append(1)   # вправо
        
        """
        Генерируем новые состояния
        Проверяем достижение цели
        """
        for delta in moves:
            new_pos = zero_pos + delta
            
            state_list = list(state)
            state_list[zero_pos], state_list[new_pos] = \
                state_list[new_pos], state_list[zero_pos]
            new_state = tuple(state_list)
            
            if new_state not in visited:
                new_path = path + [new_state]
                
                if new_state == goal:
                    return new_path, steps + 1
                
                visited.add(new_state)
                queue.append((new_state, new_path, steps + 1))
    
    return None, None

def generate_solvable_puzzle(moves_count=20):
    """
    Генерирует разрешимую головоломку (в этот момент я забыл все слова в русском языке)
    
    Делает случайные ходы от целевого состояния
    Гарантирует разрешимость результата
    """
    state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
    
    for _ in range(moves_count):
        zero_pos = state.index(0)
        row = zero_pos // 3
        col = zero_pos % 3
        
        valid_moves = []
        if row > 0: valid_moves.append(-3)
        if row < 2: valid_moves.append(3)
        if col > 0: valid_moves.append(-1)
        if col < 2: valid_moves.append(1)
        
        move = random.choice(valid_moves)
        new_pos = zero_pos + move
        state[zero_pos], state[new_pos] = state[new_pos], state[zero_pos]
    
    return state

def print_board(state): 
    """
    Печатает доску в красивом формате, работаем с визуализацией, а как иначе я пойму что все гуд?
    Пустая клетка обозначается точкой (·)
    """
    if isinstance(state, list):
        state = tuple(state)
    
    print("┌─────────┐")
    for i in range(3):
        row_str = "│"
        for j in range(3):
            val = state[i*3 + j]
            if val == 0:
                row_str += " · "
            else:
                row_str += f" {val} "
        row_str += "│"
        print(row_str)
    print("└─────────┘")

"""
ДЕМОНСТРАЦИЯ РАБОТЫ АЛГОРИТМА 
"""

if __name__ == "__main__":
    print("="*60)
    print("РЕШЕНИЕ ИГРЫ ПЯТНАШКИ 3×3 (BFS)")
    print("="*60)
    
    # Пример 1: Простая задача
    print("\n[ПРИМЕР 1] Простая головоломка:")
    start1 = [1, 2, 3, 4, 5, 6, 7, 0, 8]
    print_board(start1)
    
    solution1, steps1 = solve_puzzle(start1)
    print(f"✓ Решено за {steps1} ход(ов)\n")
    
    # Пример 2: Сложная задача
    print("[ПРИМЕР 2] Сложная головоломка:")
    random.seed(42)
    start2 = generate_solvable_puzzle(25)
    print_board(start2)
    
    solution2, steps2 = solve_puzzle(start2)
    if solution2:
        print(f"✓ Решено за {steps2} ход(ов)")
        print(f"Путь: {len(solution2)} состояний")


РЕШЕНИЕ ИГРЫ ПЯТНАШКИ 3×3 (BFS)

[ПРИМЕР 1] Простая головоломка:
┌─────────┐
│ 1  2  3 │
│ 4  5  6 │
│ 7  ·  8 │
└─────────┘
✓ Решено за 1 ход(ов)

[ПРИМЕР 2] Сложная головоломка:
┌─────────┐
│ 1  2  3 │
│ ·  4  5 │
│ 7  8  6 │
└─────────┘
✓ Решено за 3 ход(ов)
Путь: 4 состояний
